# Airflow Batch Pipeline Example

Demonstrates a simple Airflow DAG for feature engineering, training, and batch scoring.

In [1]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.utils.dates import days_ago
from datetime import timedelta

default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'bridge_failure_batch_pipeline',
    default_args=default_args,
    schedule_interval='@daily',
    start_date=days_ago(1),
    catchup=False,
) as dag:
    feat_engineering = BashOperator(
        task_id='feature_engineering',
        bash_command='python /opt/project/src/features/engineering.py'
    )

    train_model = BashOperator(
        task_id='model_training',
        bash_command='python /opt/project/src/models/train.py'
    )

    batch_score = BashOperator(
        task_id='batch_scoring',
        bash_command='python /opt/project/src/models/batch_score.py'
    )

    feat_engineering >> train_model >> batch_score